In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset
from multiprocessing import Process

/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
import imp
import sys

sys.path.insert(0, '../../src')
from utils import df_to_xarray,read_xarray, get_point_prediction, custom_rmse

sys.path.insert(0, '../../src/preprocess')
from data_preprocess import preprocess_image_reduced,preprocess_images_nfp, inverse_scale_frame

/tmp/ipykernel_1485121/3742897804.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
2022-05-31 15:09:38.528396: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [28]:
dir_name="../../data/data1"
val_dir_name="../../data/data2"

data,pco2 = preprocess_images_nfp(dir_name)
data_socat, pco2_socat = preprocess_images_nfp(dir_name, socat = True)

# val_data,val_pco2 = preprocess_images_nfp(val_dir_name,"035")
# val_data_socat,val_pco2_socat = preprocess_images_nfp(val_dir_name,"035",socat=True)

X_index=np.lib.stride_tricks.sliding_window_view(range(421),3) 

y=np.expand_dims(pco2[X_index][1:],axis=4)
X=data[X_index][:-1]

# val_y=np.expand_dims(val_pco2[X_index][1:],axis=4)
# val_X=val_data[X_index][:-1]


print(X.shape, y.shape)

INPUT_SHAPE = X[0].shape

(418, 3, 180, 360, 6) (418, 3, 180, 360, 1)


In [4]:
import torch.nn as nn

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels= 4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        
        
        input_tensor = input_tensor
        h_cur = h_cur
        
        # print("input_tensor:", input_tensor.size())
        # print("h_cur:",h_cur.size())
        # print("c_hur:",c_cur.size())
        # print("input_dimension:",self.input_dim)
        # print("hidden_dimension:",self.hidden_dim)

        combined = torch.cat([input_tensor, h_cur], dim=1) # concatenate along channel axis
        #print("combined:", combined.size())
        combined_conv = self.conv(combined)
        #print("combined_conv:", combined_conv.size())


        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = nn.ELU()(cc_g)
        
        c_next = f * c_cur + i * g
        h_next = o * nn.ELU()(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))



In [5]:
class EncoderDecoderConvLSTM(nn.Module):
    def __init__(self, nf, in_chan):
        super(EncoderDecoderConvLSTM, self).__init__()

        """ ARCHITECTURE 

        # Encoder (ConvLSTM)
        # Encoder Vector (final hidden state of encoder)
        # Decoder (ConvLSTM) - takes Encoder Vector as input
        # Decoder (3D CNN) - produces regression predictions for our model

        """
        self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)
        
        self.batch_norm = torch.nn.BatchNorm2d(nf)

        self.decoder_1_convlstm = ConvLSTMCell(input_dim=nf,  # nf + 1
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_CNN = nn.Conv3d(in_channels=nf,
                                     out_channels=3,
                                     kernel_size=(1, 3, 3),
                                     padding=(0, 1, 1))


    def autoencoder(self, x, seq_len, future_step, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4):

        outputs = []

        # encoder
        for t in range(seq_len):
            h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :],
                                               cur_state=[h_t, c_t])  # we could concat to provide skip conn here
            
            h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t,
                                                 cur_state=[h_t2, c_t2])  # we could concat to provide skip conn here

        # encoder_vector
        encoder_vector = h_t2

        # decoder
        for t in range(future_step):
            h_t3, c_t3 = self.decoder_1_convlstm(input_tensor=encoder_vector,
                                                 cur_state=[h_t3, c_t3])  # we could concat to provide skip conn here
            h_t4, c_t4 = self.decoder_2_convlstm(input_tensor=h_t3,
                                                 cur_state=[h_t4, c_t4])  # we could concat to provide skip conn here
            encoder_vector = h_t4
            outputs += [h_t4]  # predictions

        outputs = torch.stack(outputs, 1)
        outputs = outputs.permute(0, 2, 1, 3, 4)
        outputs = self.decoder_CNN(outputs)
        outputs = torch.nn.ELU()(outputs)

        return outputs

    def forward(self, x, future_seq=0, hidden_state=None):

        """
        Parameters
        ----------
        input_tensor:
            5-D Tensor of shape (b, t, c, h, w)        #   batch, time, channel, height, width
        """

        # find size of different input dimensions
        b, seq_len, _, h, w = x.size()

        # initialize hidden states
        h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t3, c_t3 = self.decoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t4, c_t4 = self.decoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))

        # autoencoder forward
        outputs = self.autoencoder(x, seq_len, future_seq, h_t, c_t, h_t2, c_t2, h_t3, c_t3, h_t4, c_t4)

        return outputs

In [6]:
from torch.utils.data import TensorDataset
from torchvision import models
from torchsummary import summary
from pytorch_lightning.callbacks import ModelCheckpoint


checkpoint_callback = ModelCheckpoint(
    "../../models",
    save_top_k=1,
    monitor="loss",
    mode="min",
    verbose=True)

batch_size = 16
lr= 0.0003
epochs = 1

/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Checkpoint directory ../../models exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


In [7]:
from pytorch_lightning import Trainer
from multiprocessing import Process
import pytorch_lightning as pl

class CESMLighting(pl.LightningModule):

    def __init__(self, hparams=None, model=None, X= None, y=None):
        super(CESMLighting, self).__init__()
        # default config
        self.normalize = False
        self.model = model

        # logging config
        self.log_images = True

        # Training config
        self.batch_size = 16
        self.epochs=100
        self.n_steps_past = 1
        self.n_steps_ahead = 1
        self.X = X
        self.y = y

    def customRmse(self,y_true, y_pred):
        """
        custom_rmse(y_true, y_pred)
        calculates root square mean value with focusing only on the ocean
        """
        y_pred = y_pred[(y_true != 0) & (y_true != 0.0)]
        y_true = y_true[(y_true != 0) & (y_true != 0.0)]
        
        y_pred = torch.Tensor.cuda(y_pred)
        y_true = y_true.type(y_pred.dtype)
        loss = torch.sqrt(torch.mean(torch.square(torch.subtract(y_pred,y_true))))
        return loss

    def forward(self, x):
        x = x.to(device='cuda')
        output = self.model(x, future_seq=self.n_steps_ahead)
        return output

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.permute(0, 1, 4, 2, 3)
        y = y.squeeze().cuda()
        y_hat = self.forward(x).squeeze()  
        
        #print("y: {} y_hat: {}".format(y.shape,y_hat.shape))

        loss = self.customRmse(y, y_hat)

        # save learning_rate
        lr_saved = self.trainer.optimizers[0].param_groups[-1]['lr']
        lr_saved = torch.scalar_tensor(lr_saved).cuda()

        tensorboard_logs = {'train_rmse_loss': loss,
                            'learning_rate': lr_saved}
        if self.global_step % 10 == 0:
            print("Current epoch {} loss: {}".format(self.current_epoch, loss.item()))
        
        return {'loss': loss, 'log': tensorboard_logs}

    def test_step(self, batch, batch_idx):
        # OPTIONAL
        x, y = batch
        x = x.permute(0, 1, 4, 2, 3)
        y_hat = self.forward(x)
        y = y.squeeze().cuda()
        y_hat = self.forward(x).squeeze() 
        return {'test_loss': self.criterion(y_hat, y)}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(),lr = lr)
    
    def train_dataloader(self):
        tensor_X = torch.Tensor(self.X) # transform to torch tensor
        tensor_y = torch.Tensor(self.y)
        cesm = TensorDataset(tensor_X,tensor_y) # create your datset

        train_loader = DataLoader(cesm,batch_size=batch_size,shuffle=False) 
        return train_loader

    def test_dataloader(self):
        tensor_X = torch.Tensor(self.X) # transform to torch tensor
        tensor_y = torch.Tensor(self.y)
        cesm = TensorDataset(tensor_X,tensor_y) # create your datset

        test_loader = DataLoader(cesm,batch_size=batch_size,shuffle=False) 
        return test_loader




In [8]:
torch.cuda.empty_cache()
conv_lstm_model = EncoderDecoderConvLSTM(nf=32, in_chan=6)
model = CESMLighting(model=conv_lstm_model,X=X,y=y)    

if torch.cuda.is_available():
    model.cuda()


trainer = Trainer(max_epochs=35,checkpoint_callback=checkpoint_callback)
trainer.fit(model)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

  | Name  | Type                   | Params
-------------------------------------------------
0 | model | EncoderDecoderConvLSTM | 266 K 
/burg/glab/users/sk4973/venv2/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Current epoch 0 loss: 77.02783966064453
Current epoch 0 loss: 79.63282775878906
Current epoch 0 loss: 79.74320983886719
Current epoch 0 loss: 57.55128860473633
Current epoch 0 loss: 24.480430603027344
Current epoch 0 loss: 16.296785354614258



Epoch 00000: loss reached 15.91725 (best 15.91725), saving model to ../../models/epoch=0.ckpt as top 2


Current epoch 1 loss: 19.806032180786133
Current epoch 1 loss: 13.998576164245605
Current epoch 1 loss: 12.664587020874023
Current epoch 1 loss: 12.021626472473145
Current epoch 1 loss: 11.473539352416992


...

Current epoch 32 loss: 4.316435813903809
Current epoch 33 loss: 3.9482688903808594
Current epoch 33 loss: 3.87135910987854
Current epoch 33 loss: 3.902240514755249
Current epoch 33 loss: 4.003685474395752
Current epoch 33 loss: 4.30327033996582



Epoch 00033: loss reached 4.31168 (best 4.31168), saving model to ../../models/epoch=33.ckpt as top 2


Current epoch 34 loss: 4.266963481903076
Current epoch 34 loss: 4.3668365478515625
Current epoch 34 loss: 4.030972003936768
Current epoch 34 loss: 4.083408832550049
Current epoch 34 loss: 3.8343653678894043



Epoch 00034: loss  was not in top 2


AttributeError: 'ellipsis' object has no attribute '__module__'

In [11]:
PATH ='../../models/pytorch_convlstm'
torch.save(model.state_dict(), PATH)

In [12]:
conv_lstm_model = EncoderDecoderConvLSTM(nf=32, in_chan=6)
model = CESMLighting(model=conv_lstm_model,X=X,y=y)   
model.load_state_dict(torch.load(PATH))
if torch.cuda.is_available():
    model.cuda()

model.eval()

CESMLighting(
  (model): EncoderDecoderConvLSTM(
    (encoder_1_convlstm): ConvLSTMCell(
      (conv): Conv2d(38, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (encoder_2_convlstm): ConvLSTMCell(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (batch_norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (decoder_1_convlstm): ConvLSTMCell(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (decoder_2_convlstm): ConvLSTMCell(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (decoder_CNN): Conv3d(32, 3, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1))
  )
)

In [56]:
tensor_X= torch.Tensor(X).cuda()
tensor_y = torch.Tensor(y).cuda()
cesm = TensorDataset(tensor_X,tensor_y) # create your datset

data_loader = DataLoader(cesm,batch_size=1,shuffle=False) 

y_pred=np.zeros(shape=([418,3,180, 360,1]))

for batch_idx, samples in enumerate(data_loader):
    s= samples[0]
    s = s.permute(0, 1, 4, 2, 3)
    #print(batch_idx, s.size())
    yp = model(s)
    yp = yp.permute(0,1,3,4,2)
    yp= yp.cpu().detach().numpy()
    y_pred[batch_idx]=yp

In [60]:
y_pred[y==0]=0.0
pred=y_pred

In [61]:
chl,mld,sss,sst,u10,xco2,icefrac,patm,pco2t2 = read_xarray(dir_name)

y_true,y_pred = inverse_scale_frame(pred,pco2t2.pCO2.data)

In [62]:
y_pred_socat = np.copy(y_pred)
y_true_socat=np.expand_dims(pco2t2.pCO2_socat.data[X_index][1:],axis=4)
y_true_socat = np.nan_to_num(y_true_socat)
y_pred_socat[y_true_socat==0]=0.0

In [64]:
print("Full RMSE score:")
a=custom_rmse(y_pred[:,:1],y_true[:,:1])
print(a.numpy())

print("SOCAT RMSE score:")
b=custom_rmse(y_pred_socat[:,:1],y_true_socat[:,:1])
print(b.numpy())

Full RMSE score:
8.116834158885355
SOCAT RMSE score:
9.008042187833174
